In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import time
import numpy as np
import cv2
import scipy.special
import matplotlib.pyplot
import os
import csv

from os import listdir
from os.path import isfile, join


In [ ]:

# neural network class definition
class neuralNetwork :
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes,learningrate,hiddenlayers) :

        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        self.hlayer = hiddenlayers
        self.bias = np.ones(self.hlayer+1)

        # link weight matrices
        self.weight = []
        self.weight.append(np.random.normal(0.0, pow(self.hnodes, -0.5),(self.hnodes, self.inodes)))

        for i in range(self.hlayer-1):
            self.weight.append(np.random.normal(0.0, pow(self.hnodes, -0.5),(self.hnodes, self.hnodes)))
            
        self.weight.append(np.random.normal(0.0, pow(self.onodes, -0.5),(self.onodes, self.hnodes)))

        # learning rate
        self.lr = learningrate

        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
    
        # train the neural network
    def train(self, inputs_list, targets_list) :
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # inputs_list = []
        hidden_list = []
        outputs_list = []
        error_list = []
        
        #calculate the signals
        hidden_list.append(self.activation_function(np.dot(self.weight[0], inputs)+self.bias[0]))
        
        for i in range(len(self.weight)):
            if i == 0:
                pass
            elif i == len(self.weight)-1:
                pass
            else:
                hidden_list.append(self.activation_function(np.dot(self.weight[i], hidden_list[i-1])+self.bias[i]))
        
        outputs_list.append(self.activation_function(np.dot(self.weight[len(self.weight)-1], hidden_list[-1])+self.bias[-1]))
        
        #keep error
        error_list.append(targets-outputs_list[-1])
        i_weight = len(self.weight)-1
        i_error   = 0
        
        while i_weight >0:
            error_list.append(np.dot(self.weight[i_weight].T, error_list[i_error]))
            i_weight = i_weight-1
            i_error    = i_error+1
        
        
        #update weight
        i_weight = len(self.weight)-1
        i_error   = 0
        i_hidden = -1

        while i_weight  > -1:
            if (i_weight == len(self.weight)-1):
                self.weight[i_weight] += self.lr * np.dot( (error_list[i_error]*outputs_list[-1] * (1.0-outputs_list[-1])) , np.transpose(hidden_list[-1])  )
                self.bias[-1] += self.lr*1*(sum(error_list[i_error]))

            else:
                if (i_weight == 0):
                    self.weight[i_weight] += self.lr * np.dot( (error_list[i_error]*hidden_list[i_hidden] * (1.0-hidden_list[i_hidden])) , np.transpose(inputs)  )
                    self.bias[i_weight] += self.lr*1*(sum(error_list[i_error]))
                    i_hidden = i_hidden-1
                else:
                    self.weight[i_weight] += self.lr * np.dot( (error_list[i_error]*hidden_list[i_hidden] * (1.0-hidden_list[i_hidden])) , np.transpose(hidden_list[i_hidden-1])  )
                    self.bias[i_weight] += self.lr*1*(sum(error_list[i_error]))
                    i_hidden = i_hidden-1
            i_weight = i_weight-1
            i_error   = i_error+1

        # print(self.weight)
        # print("***********************")
        pass
    
    def query(self, inputs_list) :

        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T

        
        hidden_list = []
        outputs_list = []
        error_list = []
        
        #calculate the signals
        hidden_list.append(self.activation_function(np.dot(self.weight[0], inputs)+self.bias[0]))
        
        for i in range(len(self.weight)):
            if i == 0:
                pass
            elif i == len(self.weight)-1:
                pass
            else:
                hidden_list.append(self.activation_function(np.dot(self.weight[i], hidden_list[i-1])+self.bias[i]))
        
        outputs_list.append(self.activation_function(np.dot(self.weight[len(self.weight)-1], hidden_list[-1])+self.bias[-1]))

        return outputs_list[-1]
        





In [ ]:
def histrogam_to_32_bins(image_file):
    img = cv2.imread(image_file)
    if(len(img[0])>len(img[0][0])):
        img = np.rot90(img)
    img = cv2.resize(img, (150,180))
    ##cv2.imshow("img",img)


    all_r_list = []
    all_g_list = []
    all_b_list = []

    for row in range(len(img)):
        for col in range(len(img[0])):
            all_r_list.append(img[row][col][0])
            all_g_list.append(img[row][col][1])
            all_b_list.append(img[row][col][2])

    ##r_list = np.unique(all_r_list,axis=0)
    ##g_list = np.unique(all_g_list,axis=0)
    ##b_list = np.unique(all_b_list,axis=0)

    r_list = np.zeros(256)
    g_list = np.zeros(256)
    b_list = np.zeros(256)


    for val in all_r_list:
        r_list[val] = r_list[val]+1
    for val in all_g_list:
        g_list[val] = g_list[val]+1
    for val in all_b_list:
        b_list[val] = b_list[val]+1

    r_bins = []
    g_bins = []
    b_bins = []
    
    for range_bins in range(256//8):
        
        r_bins.append(sum(r_list[range_bins*8:range_bins*8+8]))
        g_bins.append(sum(g_list[range_bins*8:range_bins*8+8]))
        b_bins.append(sum(b_list[range_bins*8:range_bins*8+8]))

    return r_bins,g_bins,b_bins

In [ ]:

input_nodes = 96
hidden_nodes = 48
output_nodes = 4
hiddenlayers = 1

# learning rate
learning_rate = 0.5

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate,hiddenlayers)

# load the mnist training data CSV file into a list
train_data_list = []
with open('drive/My Drive/korat_sauce/img_csv.csv') as f_img:
    img_bins = csv.reader(f_img)
    for im in img_bins:
        train_data_list.append(im)
    f_img.close()

# train the neural network
# epochs is the number of times the training data set is used for training
epochs = 5
for e in range(epochs):
    # go through all records in the training data set
    for record in train_data_list:
        
        # scale and shift the inputs
        inputs = (np.asfarray(record[1:]) / 255.0 * 0.99) +0.01
        
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = np.zeros(output_nodes) + 0.01
        
        # all_values[0] is the target label for this record
        targets[int(record[0])-1] = 0.99 # -1 beacause use class 1,2,3,4 but index in target array is 0,1,2,3
        n.train(inputs, targets)
        pass
    pass



In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/test_all_sauce4.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/My Drive/korat_sauce/test_all_sauce4.zip
   creating: test_all_sauce/
  inflating: test_all_sauce/maeyupa.0.jpg  
  inflating: test_all_sauce/maeyupa.1.jpg  
  inflating: test_all_sauce/maeyupa.10.jpg  
  inflating: test_all_sauce/maeyupa.11.jpg  
  inflating: test_all_sauce/maeyupa.12.jpg  
  inflating: test_all_sauce/maeyupa.13.jpg  
  inflating: test_all_sauce/maeyupa.14.jpg  
  inflating: test_all_sauce/maeyupa.15.jpg  
  inflating: test_all_sauce/maeyupa.16.jpg  
  inflating: test_all_sauce/maeyupa.17.jpg  
  inflating: test_all_sauce/maeyupa.18.jpg  
  inflating: test_all_sauce/maeyupa.19.jpg  
  inflating: test_all_sauce/maeyupa.2.jpg  
  inflating: test_all_sauce/maeyupa.20.jpg  
  inflating: test_all_sauce/maeyupa.21.jpg  
  inflating: test_all_sauce/maeyupa.22.jpg  
  inflating: test_all_sauce/maeyupa.23.jpg  
  inflating: test_all_s

In [ ]:
mypath_test = 'test_all_sauce/'
imagePaths_test = [mypath_test+f for f in listdir(mypath_test) if isfile(join(mypath_test, f))]

In [ ]:
#######
accuracy_pack_test = 0
for (i, img_path) in enumerate(imagePaths_test):
  class_of_test = img_path.split(".")[0].split("/")[1]

  r_bins,g_bins,b_bins = histrogam_to_32_bins(img_path)
  img_list = list(r_bins)+list(g_bins)+list(b_bins)
  img_list = (np.asfarray(record[1:]) / 255.0 * 0.99) +0.01
  
  outputs = n.query(inputs)
  label = np.argmax(outputs)
  print('label = ',label)
  print('class_of_test = ',class_of_test)
  print('***************')
  
  if(class_of_test=="maeyupa"):
    if(label == 0):
      accuracy_pack_test+=1

  elif(class_of_test=="ocean"):
    if(label == 1):
      accuracy_pack_test+=1

  elif(class_of_test=="zapnuer"):
    if(label == 2):
      accuracy_pack_test+=1

  elif(class_of_test=="zapnuerchick"):
    if(label == 3):
      accuracy_pack_test+=1
  # print("#####")

print("accuracy = ",accuracy_pack_test,"/",i+1," = ",accuracy_pack_test/(i+1)*100)

label =  2
class_of_test =  zapnuerchick
***************
label =  2
class_of_test =  zapnuerchick
***************
label =  2
class_of_test =  zapnuerchick
***************
label =  2
class_of_test =  zapnuer
***************
label =  2
class_of_test =  ocean
***************
label =  2
class_of_test =  zapnuer
***************
label =  2
class_of_test =  maeyupa
***************
label =  2
class_of_test =  zapnuerchick
***************
label =  2
class_of_test =  zapnuer
***************
label =  2
class_of_test =  zapnuer
***************
label =  2
class_of_test =  zapnuer
***************
label =  2
class_of_test =  zapnuerchick
***************
label =  2
class_of_test =  zapnuer
***************
label =  2
class_of_test =  zapnuer
***************
label =  2
class_of_test =  ocean
***************
label =  2
class_of_test =  maeyupa
***************
label =  2
class_of_test =  maeyupa
***************
label =  2
class_of_test =  maeyupa
***************
label =  2
class_of_test =  zapnuer
*******

In [ ]:
a = np.array([1,2,3])
b = np.array([4,5,6])
c = []

c.append(a)
c.append(b)

activation_function = lambda x: scipy.special.expit(x)
d = []
d.append(activation_function(a))
b-a